<a href="https://colab.research.google.com/github/farzadkhazaee72/machinelearning/blob/master/Heat_Transfer_Simulation_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import math
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

# Dashboard Inputs

In [ ]:
# User inputs
mission_name = "S-Z1-S"
compressor_power = 15
number_of_pallets = 26
thermostat_range = 3                 # Thermostat Range (+/-)
is_waste_heat_used = True            # Is waste heat from Fuel Cell used in heating mode?
fuel_cell_eff = 0.58                 # Fuel Cell efficiency (%)
fuel_cell_bop = 1000                 # Fuel Cell Balance of Plant W
fuel_cell_fan = 0.3                  # TRU fan power kW
start_time_str = "01/01/23 9:00:00"
time_delta = 5                       # in seconds

# Mission

In [ ]:
class Mission:
    def __init__(self, name="B-Z1-W"):
        self.name = name

    def get_values(self):
        match self.name:
            case "B-Z1-W":
                mission_cargo = "Bananas (fresh)"
                mission_zone = "Zone 1"
                mission_season = "Winter"
            case "B-Z2-W":
                mission_cargo = "Bananas (fresh)"
                mission_zone = "Zone 2"
                mission_season = "Winter"
            case "B-Z3-W":
                mission_cargo = "Bananas (fresh)"
                mission_zone = "Zone 3"
                mission_season = "Winter"
            case "B-Z1-S":
                mission_cargo = "Bananas (fresh)"
                mission_zone = "Zone 1"
                mission_season = "Summer"
            case "B-Z2-S":
                mission_cargo = "Bananas (fresh)"
                mission_zone = "Zone 2"
                mission_season = "Summer"
            case "B-Z3-S":
                mission_cargo = "Bananas (fresh)"
                mission_zone = "Zone 3"
                mission_season = "Summer"
            case "O-Z1-W":
                mission_cargo = "Oranges (fresh)"
                mission_zone = "Zone 1"
                mission_season = "Winter"
            case "O-Z2-W":
                mission_cargo = "Oranges (fresh)"
                mission_zone = "Zone 2"
                mission_season = "Winter"
            case "O-Z3-W":
                mission_cargo = "Oranges (fresh)"
                mission_zone = "Zone 3"
                mission_season = "Winter"
            case "O-Z1-S":
                mission_cargo = "Oranges (fresh)"
                mission_zone = "Zone 1"
                mission_season = "Summer"
            case "O-Z2-S":
                mission_cargo = "Oranges (fresh)"
                mission_zone = "Zone 2"
                mission_season = "Summer"
            case "O-Z3-S":
                mission_cargo = "Oranges (fresh)"
                mission_zone = "Zone 3"
                mission_season = "Summer"
            case "S-Z1-W":
                mission_cargo = "Shrimp (frozen)"
                mission_zone = "Zone 1"
                mission_season = "Winter"
            case "S-Z2-W":
                mission_cargo = "Shrimp (frozen)"
                mission_zone = "Zone 2"
                mission_season = "Winter"
            case "S-Z3-W":
                mission_cargo = "Shrimp (frozen)"
                mission_zone = "Zone 3"
                mission_season = "Winter"
            case "S-Z1-S":
                mission_cargo = "Shrimp (frozen)"
                mission_zone = "Zone 1"
                mission_season = "Summer"
            case "S-Z2-S":
                mission_cargo = "Shrimp (frozen)"
                mission_zone = "Zone 2"
                mission_season = "Summer"
            case "S-Z3-S":
                mission_cargo = "Shrimp (frozen)"
                mission_zone = "Zone 3"
                mission_season = "Summer"
            case _:
                mission_cargo = "Bananas (fresh)"
                mission_zone = "Zone 1"
                mission_season = "Winter"

        return mission_cargo, mission_zone, mission_season

# Initialize mission
mission = Mission(name=mission_name)
mission_cargo, mission_zone, mission_season = mission.get_values()

In [ ]:
print(mission_cargo, mission_zone, mission_season)

Shrimp (frozen) Zone 1 Summer


# Trailer

In [ ]:
L_o_imperial = 636  # Outer length in feet converted to inches (53x12)
W_o_imperial = 102  # Outer width in inches
H_o_imperial = 110  # Outer height in inches

L_i_imperial = 630  # Inner length in feet converted to inches (52.5x12)
W_i_imperial = 96   # Inner width in inches
H_i_imperial = 104  # Inner height in inches

L_o = round(L_o_imperial*25.4/1000, 4)   # Outer length m
W_o = round(W_o_imperial*25.4/1000, 4)   # Outer width m
H_o = round(H_o_imperial*25.4/1000, 4)   # Outer height m

L_i = round(L_i_imperial*25.4/1000, 4)   # Inner length m
W_i = round(W_i_imperial*25.4/1000, 4)   # Inner width m
H_i = round(H_i_imperial*25.4/1000, 4)   # Inner height m

A_1_o = round(W_o*H_o, 4)  # Outer surface area - Front m2
A_2_o = round(W_o*H_o, 4)  # Outer surface area - Back  m2
A_3_o = round(L_o*H_o, 4)  # Outer surface area - Left m2
A_4_o = round(L_o*H_o, 4)  # Outer surface area - Right m2
A_5_o = round(L_o*W_o, 4)  # Outer surface area - Top m2
A_6_o = round(L_o*W_o, 4)  # Outer surface area - Bottom m2

A_total_o = round(A_1_o + A_2_o + A_3_o + A_4_o + A_5_o + A_6_o, 4)   # Total Area - Outer m2
V_o = round(L_o * W_o * H_o, 4)                                       # Trailer Volume - Outer m3

A_1_i = round(W_i*H_i, 4)   # Inner surface area - Front m2
A_2_i = round(W_i*H_i, 4)   # Inner surface area - Back m2
A_3_i = round(L_i*H_i, 4)   # Inner surface area - Left m2
A_4_i = round(L_i*H_i, 4)   # Inner surface area - Right = **** m2
A_5_i = round(L_i*W_i, 4)   # Inner surface area - Top m2
A_6_i = round(L_i*W_i, 4)   # Inner surface area - Bottom m2

A_total_i = round(A_1_i + A_2_i + A_3_i + A_4_i + A_5_i + A_6_i, 4)   # Total Area - Inner = m2
V_i = round(L_i * W_i * H_i, 4)                                       # Trailer Volume - Inner m3

R_1, R_2, R_3, R_4, R_5, R_6 = 10, 10, 10, 10, 10, 10
RSI_1, RSI_2, RSI_3, RSI_4, RSI_5, RSI_6 = round(R_1/5.678, 9), round(R_2/5.678, 9), round(R_3/5.678, 9), \
                                           round(R_4/5.678, 9), round(R_5/5.678, 9), round(R_6/5.678, 9)


U_1 = round(1/RSI_1, 4)  # Thermal transmittance - Front W/(Km2)
U_2 = round(1/RSI_2, 4)  # Thermal transmittance - Back W/(Km2)
U_3 = round(1/RSI_3, 4)  # Thermal transmittance - Left W/(Km2)
U_4 = round(1/RSI_4, 4)  # Thermal transmittance - Right W/(Km2)
U_5 = round(1/RSI_5, 4)  # Thermal transmittance - Top W/(Km2)
U_6 = round(1/RSI_6, 4)  # Thermal transmittance - Bottom W/(Km2)


print(L_o, W_o, H_o)
print(L_i, W_i, H_i)
print(A_1_o, A_2_o, A_3_o, A_4_o, A_5_o, A_6_o, A_total_o, V_o)
print(A_1_i, A_2_i, A_3_i, A_4_i, A_5_i, A_6_i, A_total_i, V_i)
print(U_1, U_2, U_3, U_4, U_5, U_6)

16.1544 2.5908 2.794
16.002 2.4384 2.6416
7.2387 7.2387 45.1354 45.1354 41.8528 41.8528 188.4538 116.9368
6.4413 6.4413 42.2709 42.2709 39.0193 39.0193 175.463 103.0733
0.5678 0.5678 0.5678 0.5678 0.5678 0.5678


# Cargo

In [ ]:
class Cargo:
    def __init__(self, number_of_pallets, cargo_type="Bananas (fresh)"):
        self.cargo_type = cargo_type
        self.number_of_pallets = number_of_pallets

    def get_cargo_details(self):
        density, spec_heat_cap, box_height, box_width, box_length, box_mass, box_volume, stack_box_height, \
        stack_box_width, stack_box_length, boxes_per_stack, stack_front_area, stack_side_area, stack_top_area, \
        stack_area, box_type, surface_area_factor, volume_factor, ideal_temperature = self.get_cargo_type_data()

        # round the values to 2 decimal
        stack_area = round(stack_area, 2)
        surface_area_factor = round(surface_area_factor, 2)
        volume_factor = round(volume_factor, 2)

        total_boxes = self.number_of_pallets * boxes_per_stack
        surface_area_of_cargo = round(surface_area_factor * self.number_of_pallets * stack_area, 2)   # A_c
        volume_of_cargo = round(volume_factor * total_boxes * box_volume, 2)                          # V_c
        mass_of_cargo = round(total_boxes * box_mass, 2)                                              # m_c
        heat_cpacity_of_cargo = round(mass_of_cargo * spec_heat_cap, 2)                               # sigma_c

        return [self.cargo_type, self.number_of_pallets, boxes_per_stack, total_boxes, box_type,
                stack_area, surface_area_factor, surface_area_of_cargo, box_volume, volume_factor,
                volume_of_cargo, mass_of_cargo, spec_heat_cap, heat_cpacity_of_cargo]

    def get_cargo_type_data(self):
        match self.cargo_type:
            case "Oranges (fresh)":
                density = 958
                spec_heat_cap = 3770
                box_height = 10*2.54/100
                box_width = 20*2.54/100
                box_length = 16*2.54/100
                box_mass = 15
                box_volume = box_height * box_width * box_length
                stack_box_height = 8
                stack_box_width = 2
                stack_box_length = 3
                boxes_per_stack = stack_box_height * stack_box_width * stack_box_length
                stack_front_area = box_height * stack_box_height * box_width * stack_box_width
                stack_side_area = box_height * stack_box_height * box_length * stack_box_length
                stack_top_area = box_width * stack_box_width * box_length * stack_box_length
                stack_area = 2 * (stack_front_area + stack_side_area + stack_top_area)
                box_type = "vented"
                surface_area_factor = 1 if box_type == "sealed" else 2
                volume_factor = box_mass/(density * box_volume) if box_type == "vented" else 1
                ideal_temperature = 2

            case "Bananas (fresh)":
                density = 940
                spec_heat_cap = 3350
                box_height = 10*2.54/100
                box_width = 20*2.54/100
                box_length = 16*2.54/100
                box_mass = 15
                box_volume = box_height * box_width * box_length
                stack_box_height = 8
                stack_box_width = 2
                stack_box_length = 3
                boxes_per_stack = stack_box_height * stack_box_width * stack_box_length
                stack_front_area = box_height * stack_box_height * box_width * stack_box_width
                stack_side_area = box_height * stack_box_height * box_length * stack_box_length
                stack_top_area = box_width * stack_box_width * box_length * stack_box_length
                stack_area = 2 * (stack_front_area + stack_side_area + stack_top_area)
                box_type = "vented"
                surface_area_factor = 1 if box_type == "sealed" else 2
                volume_factor = box_mass/(density * box_volume) if box_type == "vented" else 1
                ideal_temperature = 14

            case "Shrimp (frozen)":
                density = 1017
                spec_heat_cap = 1890
                box_height = 10*2.54/100
                box_width = 20*2.54/100
                box_length = 16*2.54/100
                box_mass = 15
                box_volume = box_height * box_width * box_length
                stack_box_height = 8
                stack_box_width = 2
                stack_box_length = 3
                boxes_per_stack = stack_box_height * stack_box_width * stack_box_length
                stack_front_area = box_height * stack_box_height * box_width * stack_box_width
                stack_side_area = box_height * stack_box_height * box_length * stack_box_length
                stack_top_area = box_width * stack_box_width * box_length * stack_box_length
                stack_area = 2 * (stack_front_area + stack_side_area + stack_top_area)
                box_type = "sealed"
                surface_area_factor = 1 if box_type == "sealed" else 2
                volume_factor = box_mass/(density * box_volume) if box_type == "vented" else 1
                ideal_temperature = -30

        return [density, spec_heat_cap, box_height, box_width, box_length, box_mass, box_volume, stack_box_height,
                stack_box_width, stack_box_length, boxes_per_stack, stack_front_area, stack_side_area, stack_top_area,
                stack_area, box_type, surface_area_factor, volume_factor, ideal_temperature]




### Cargo Parameters
- Convection coefficient at surface of cargo = **h_c** W/(Km2)
- Surface area of cargo =	**A_c**	 m2
- Volume of cargo (i.e., displaced air) = **V_c**	 m3
- Heat capacity of cargo = **sigma_c** J/K
- Time constant (cargo) = **alpha_c**	1/s

In [ ]:
cargo = Cargo(number_of_pallets, cargo_type=mission_cargo)

"""Cargo parameters"""
cargo_type, number_of_pallets, boxes_per_stack, total_boxes, box_type, \
stack_area, surface_area_factor, surface_area_of_cargo, box_volume, volume_factor, \
volume_of_cargo, mass_of_cargo, spec_heat_cap, heat_cpacity_of_cargo = cargo.get_cargo_details()

"""Content specific parameters"""
density, spec_heat_cap, box_height, box_width, box_length, box_mass, box_volume, stack_box_height, \
stack_box_width, stack_box_length, boxes_per_stack, stack_front_area, stack_side_area, stack_top_area, \
stack_area, box_type, surface_area_factor, volume_factor, ideal_temperature = cargo.get_cargo_type_data()

h_c = 10                          # Convection coefficient at surface of cargo W/(Km2)
A_c = surface_area_of_cargo       # Surface area of cargo m2
V_c = volume_of_cargo             # Volume of cargo (i.e., displaced air) m3
sigma_c = heat_cpacity_of_cargo   # Heat capacity of cargo J/K
alpha_c = h_c*A_c/sigma_c         # Time constant (cargo) 1/s

print("cargo type:", cargo_type, "number_of_pallets:",number_of_pallets, "boxes_per_stack:", boxes_per_stack,
      "total_boxes:",total_boxes, "box_type:", box_type, "stack_area:", stack_area,
      "surface_area_factor:", surface_area_factor, "surface_area_of_cargo", surface_area_of_cargo,
      "box_volume:", box_volume, "volume_factor", volume_factor, "volume_of_cargo:", volume_of_cargo,
      "mass_of_cargo:", mass_of_cargo, "spec_heat_cap:", spec_heat_cap, "heat_cpacity_of_cargo", heat_cpacity_of_cargo)



print("density:", density, "spec_heat_cap:", spec_heat_cap, "box_height:",box_height,
      "box_width:", box_width, "box_length", box_length, "box_mass", box_mass, "box_volume", box_volume,
      "stack_box_height:", stack_box_height, "stack_box_width:",stack_box_width,
      "stack_box_length:", stack_box_length, "boxes_per_stack:", boxes_per_stack,
      "stack_front_area:",stack_front_area, "stack_side_area:", stack_side_area,
      "stack_top_area", stack_top_area, "stack_area:", stack_area, "box_type",box_type,
      "surface_area_factor:", surface_area_factor, "volume_factor", volume_factor,
      "ideal_temperature", ideal_temperature)


print(h_c, A_c, V_c, sigma_c, alpha_c)

cargo type: Shrimp (frozen) number_of_pallets: 26 boxes_per_stack: 48 total_boxes: 1248 box_type: sealed stack_area: 11.5612672 surface_area_factor: 1 surface_area_of_cargo 300.56 box_volume: 0.052438604800000004 volume_factor 1 volume_of_cargo: 65.44 mass_of_cargo: 18720 spec_heat_cap: 1890 heat_cpacity_of_cargo 35380800
density: 1017 spec_heat_cap: 1890 box_height: 0.254 box_width: 0.508 box_length 0.4064 box_mass 15 box_volume 0.052438604800000004 stack_box_height: 8 stack_box_width: 2 stack_box_length: 3 boxes_per_stack: 48 stack_front_area: 2.064512 stack_side_area: 2.4774144 stack_top_area 1.2387072 stack_area: 11.5612672 box_type sealed surface_area_factor: 1 volume_factor 1 ideal_temperature -30
10 300.56 65.44 35380800 8.495002939447384e-05


### Air Volume Parameters

- Density of air = **rho_v** kg/m3
- Specific heat capacity of air volume = **C_v** J/(K kg)
- Volume of air = **V_v** m3
- Heat capacity of air volume =	**sigma_v**	J/K
- Ratio of heat capacities = **beta**

In [ ]:
rho_v = 1.1                             # Density of air kg/m3
C_v = 1000                              # Specific heat capacity of air volume J/(K kg)
V_v = round(V_i - V_c, 4)               # Volume of air m3
sigma_v = round(rho_v * V_v * C_v, 2)   # Heat capacity of air volume J/K
beta = round(sigma_c/sigma_v, 2)        # Ratio of heat capacities

print(rho_v, C_v, V_v, sigma_v, beta)

1.1 1000 37.6333 41396.63 854.68


# Zone

In [ ]:
class Zone:
    def __init__(self, name="Zone 1"):
        self.name = name

    def get_values(self):
        match self.name:
            case "Zone 1":
                route = "LA to Los Vegas"
                temp_winter = 1   # in C
                temp_summer = 42   # in C
                duration = "5:39:00"
                include_wind = True
                include_sol_air = False
                include_road_sol_air = False

            case "Zone 2":
                route = "Los Vegas to Denver"
                temp_winter = -11   # in C
                temp_summer = 42   # in C
                duration = "14:29:00"
                include_wind = True
                include_sol_air = False
                include_road_sol_air = False

            case "Zone 3":
                route = "Denver to Chicago"
                temp_winter = -23   # in C
                temp_summer = 37   # in C
                duration = "19:03:00"
                include_wind = True
                include_sol_air = False
                include_road_sol_air = False

            case _:
                route = "LA to Los Vegas"
                temp_winter = 1   # in C
                temp_summer = 42   # in C
                duration = "5:39:00"
                include_wind = True
                include_sol_air = False
                include_road_sol_air = False

        return route, temp_winter, temp_summer, duration, include_wind, include_sol_air, include_road_sol_air

# Initialize Zone
zone = Zone(name=mission_zone)
route, temp_winter, temp_summer, duration_str, include_wind, include_sol_air, include_road_sol_air = zone.get_values()

In [ ]:
# Zone temperature conditions
if mission_season == "Winter":
    temp_hi = temp_winter    # Temp Hi in C
    temp_lo = temp_winter    # Temp Lo in C
    temp_av = temp_winter    # Temp Av in C
    t_peak = 12	             # Peak time in hours
else:
    temp_hi = temp_summer    # Temp Hi in C
    temp_lo = temp_summer    # Temp Lo in C
    temp_av = temp_summer    # Temp Av in C
    t_peak = 12	             # Peak time in hours

In [ ]:
# Set the initial TRU mode
if temp_av > ideal_temperature:
    TRU_MODE = "Cooling"
elif temp_av < ideal_temperature:
    TRU_MODE = "Heating"
else:
    TRU_MODE = "Off"

# Ambience Simulation

### Ambient Parameters
- Temp Hi = **temp_hi** C
- Temp Lo = **temp_lo** C
- Temp Av = **temp _av** C
- Peak time = **t_peak** hours
- Time Increment = **t_inc** hours

In [ ]:
class AmbienceSimulator:
    def get_temperature(self, time_h):
        T_Ambient = temp_av + 0.5 * (temp_hi - temp_lo) * math.cos(2 * math.pi * (time_h - t_peak) / 24)
        return round(T_Ambient, 2)

# Initialize Ambient conditions
siml_ambience = AmbienceSimulator()

# Convection

### Convection Parameters at speed

- Natural convection coefficient - Air = **h_0** W/(Km2)
- Forced convection coefficient - Air - Max = **h_max** W/(Km2)
- "Transition" air speed (from linear to max) = **s_max** m/s
- Air Speed = **air_speed**
- Convection coefficient = **h** = if(air_speed>s_max,h_max,h_0+(h_max-h_0)*(air_speed/s_max))

In [ ]:
class Convection:
    h_0 = 5         # Natural convection coefficient - Air in W/(Km2)
    h_max = 100     # Forced convection coefficient - Air - Max in W/(Km2)
    s_max = 10      # Transition air speed (from linear to max) m/s

    def calculate_h(self, air_speed):
        coef = self.h_max if air_speed > self.s_max else self.h_0 + (self.h_max - self.h_0 ) * (air_speed/self.s_max)
        return round(coef, 2)

# Initialize Convection
siml_convection = Convection()

# Wind Simulation

In [ ]:
class WindSimulator:
    def get_air_speed(self, time_h):
        air_speed_av = 100
        air_speed_hi = 100
        air_speed_lo = 100
        t_peak = 12

        air_speed = air_speed_av + 0.5 * (air_speed_hi - air_speed_lo) * math.cos(2 * math.pi * (time_h - t_peak) / 24)
        return air_speed

# Initialize Wind conditions
siml_wind = WindSimulator()

In [ ]:
# test h values at different timestamp
# for time_h in np.arange(0.0, 2.0,0.25):
#     air_speed = siml_wind.get_air_speed(time_h)
#     convection = Convection()
#     h = convection.calculate_h(air_speed)
#     print(h)

# Radiation Simulation

In [ ]:
class SolarAndRoadRadiationSimulator:
    def __get_air_speed(self, time_h):
        """Private method for calculating air speed """
        air_speed = siml_wind.get_air_speed(time_h)
        return air_speed

    def __get_h(self, air_speed):
        """Private method for calculating h """
        h = siml_convection.calculate_h(air_speed)
        return h

    def __calculate_gamma_values(self, time_h):
        """Private method for calculating gamma values """
        air_speed = self.__get_air_speed(time_h)
        h = self.__get_h(air_speed)
        gamma_1 = h/(h+U_1) # GAMMA Front
        gamma_2 = h/(h+U_2) # GAMMA Back
        gamma_3 = h/(h+U_3) # GAMMA Left
        gamma_4 = h/(h+U_4) # GAMMA Right
        gamma_5 = h/(h+U_5) # GAMMA Top
        gamma_6 = h/(h+U_6) # GAMMA Bottom

        # print("gamma values: ", gamma_1, gamma_2, gamma_3, gamma_4, gamma_5, gamma_6)
        return gamma_1, gamma_2, gamma_3, gamma_4, gamma_5, gamma_6

    def __calculate_k_values(self, time_h):
        """Private method for calculating k values """
        gamma_1, gamma_2, gamma_3, gamma_4, gamma_5, gamma_6 = self.__calculate_gamma_values(time_h)
        k_1 = U_1*gamma_1  # K Front
        k_2 = U_2*gamma_2  # K Back
        k_3 = U_3*gamma_3  # K Left
        k_4 = U_4*gamma_4  # K Right
        k_5 = U_5*gamma_5  # K Top
        k_6 = U_6*gamma_6  # K Bottom

        # print("k values: ", k_1, k_2, k_3, k_4, k_5, k_6)
        return k_1, k_2, k_3, k_4, k_5, k_6

    def __calculate_alpha_values(self, time_h):
        """Private method for calculating alpha values """
        k_1, k_2, k_3, k_4, k_5, k_6 = self.__calculate_k_values(time_h)
        alpha_1 = k_1*A_1_o/sigma_v   # ALPHA Front
        alpha_2 = k_2*A_2_o/sigma_v   # ALPHA Back
        alpha_3 = k_3*A_3_o/sigma_v   # ALPHA Left
        alpha_4 = k_4*A_4_o/sigma_v   # ALPHA Right
        alpha_5 = k_5*A_5_o/sigma_v   # ALPHA Top
        alpha_6 = k_6*A_6_o/sigma_v   # ALPHA Bottom

        # print("alpha values: ", alpha_1, alpha_2, alpha_3, alpha_4, alpha_5, alpha_6)
        return alpha_1, alpha_2, alpha_3, alpha_4, alpha_5, alpha_6

    def get_alpha_tot(self, time_h):
        """Public method for alpha_tot used in other calculations """
        alpha_1, alpha_2, alpha_3, alpha_4, alpha_5, alpha_6 = self.__calculate_alpha_values(time_h)
        alpha_tot = alpha_1 + alpha_2 + alpha_3 + alpha_4 + alpha_5 + alpha_6  # ALPHA Total
        return alpha_tot

    def __calculate_H_rad_values(self, time_h):
        alpha_1, alpha_2, alpha_3, alpha_4, alpha_5, alpha_6 = self.__calculate_alpha_values(time_h)
        gamma_1, gamma_2, gamma_3, gamma_4, gamma_5, gamma_6 = self.__calculate_gamma_values(time_h)

        # Check if solar and road radiations are turned ON/OFF above
        T_RAD_SUN = 10 if include_sol_air else 0
        T_RAD_ROAD = 5 if include_road_sol_air else 0
        H_rad_1 = 0                               # HEAT FROM SOL-AIR TEMPERATURE Front
        H_rad_2 = 0                               # HEAT FROM SOL-AIR TEMPERATURE Back
        H_rad_3 = 0                               # HEAT FROM SOL-AIR TEMPERATURE Left
        H_rad_4 = 0                               # HEAT FROM SOL-AIR TEMPERATURE Right
        H_rad_5 = (alpha_5/gamma_5)*T_RAD_SUN     # HEAT FROM SOL-AIR TEMPERATURE Top
        H_rad_6 = (alpha_6/gamma_6)*T_RAD_ROAD    # HEAT FROM SOL-AIR TEMPERATURE Bottom

        # print("H_rad values: ", H_rad_1, H_rad_2, H_rad_3, H_rad_4, H_rad_5, H_rad_6)
        return H_rad_1, H_rad_2, H_rad_3, H_rad_4, H_rad_5, H_rad_6

    def get_H_rad_tot(self, time_h):
        """Public method for H_rad_tot used in other calculations """
        H_rad_1, H_rad_2, H_rad_3, H_rad_4, H_rad_5, H_rad_6 = self.__calculate_H_rad_values(time_h)
        H_rad_tot = H_rad_1 + H_rad_2 + H_rad_3 + H_rad_4 + H_rad_5 + H_rad_6  # HEAT FROM SOL-AIR TEMPERATURE Total
        return H_rad_tot


# Initialize solar and road radiation
siml_radiation = SolarAndRoadRadiationSimulator()

In [ ]:
#test alpha_tot and H_rad_tot values at different timestamp
for time_h in np.arange(0.0, 2.0,0.25):
    alpha_tot = siml_radiation.get_alpha_tot(time_h)
    print("alpha_tot:  ", alpha_tot)
    H_rad_tot = siml_radiation.get_H_rad_tot(time_h)
    print("H_rad_tot:  ", H_rad_tot)

alpha_tot:   0.0025702558117417676
H_rad_tot:   0.0
alpha_tot:   0.0025702558117417676
H_rad_tot:   0.0
alpha_tot:   0.0025702558117417676
H_rad_tot:   0.0
alpha_tot:   0.0025702558117417676
H_rad_tot:   0.0
alpha_tot:   0.0025702558117417676
H_rad_tot:   0.0
alpha_tot:   0.0025702558117417676
H_rad_tot:   0.0
alpha_tot:   0.0025702558117417676
H_rad_tot:   0.0
alpha_tot:   0.0025702558117417676
H_rad_tot:   0.0


# Time Window

### Time Parameters

- Delta t = **delta** seconds
- Duration = **duration_hours** hours
- Start time (24 hour clock) = **start_time** hours (on 24 hour clock)

In [ ]:
class SimulationTimeWindow:
    def __init__(self, start_time_str, duration_str):
        self.start_time = datetime.strptime(start_time_str, '%m/%d/%y %H:%M:%S')
        self.start_time_hours = self.start_time.hour  # hours (on 24 hour clock)

        self.duration = datetime.strptime(duration_str, '%H:%M:%S').time()
        self.duration_hours = self.duration.hour + (self.duration.minute/60) + (self.duration.second/3600) # total hours

        self.end_time = self.start_time + timedelta(hours=self.duration.hour,
                                                    minutes=self.duration.minute,
                                                    seconds=self.duration.second)


# Initialize Time window
siml_time_window = SimulationTimeWindow(start_time_str, duration_str)

# print(stw.start_time)
# print(stw.start_time_hours)
# print(stw.duration_hours)
# print(stw.end_time)

# Clock

In [ ]:
class Clock:
    def __init__(self, delta, init_clock_h):
        self.delta = delta # seconds
        self.init_clock_h = init_clock_h

    def update_clock_time(self, last_time_s):
        time_s = last_time_s + self.delta
        time_h =  round(time_s/3600, 2)
        clock_h = self.init_clock_h + time_h

        return time_s, time_h, clock_h


# Initialize Clock
init_clock_h = siml_time_window.start_time_hours
clck = Clock(time_delta, init_clock_h)

In [ ]:
# Update clock time after every iteration
# time_s = 0
# time_h = 0
# for i in range(50):
#     time_s, time_h, clock_h = clck.update_clock_time(time_s)
#     print(time_s, time_h, clock_h)

# Temperatures

- Ambient Temperature = **T_Ambient**
- Cargo	Temperature = **T_Cargo** = **T_Cargo + dot_T_C * time_s**
- Air Volume Temperature = **T_Air_Volume** = **T_Air_Volume + dot_T_V * time_s**

- **dot_T_C** = **alpha_c \* (T_Air_Volume - T_Cargo)**
- **dot_T_V** = **Term 7.1 + Term 7.2 + Term 7.3 + Term 7.4** from Equation 7

- **Term 7.1** = **alpha_tot * (T_Ambient - T_Air_Volume)**
- **Term 7.2** = **H_rad_tot**
- **Term 7.3** = **(phi_U + phi_O) / sigma_v**
- **Term 7.4** = **-beta * dot_T_C**


In [ ]:
# Ambient temperature
def calculate_ambient_temp(time_h):
    T_Ambient = siml_ambience.get_temperature(time_h)
    return T_Ambient

In [ ]:
# Cargo temperature
def calculate_cargo_temp(T_Cargo_prev, dot_T_C, time_delta):
    T_Cargo = T_Cargo_prev + dot_T_C * time_delta
    return T_Cargo

In [ ]:
# Air volume temperature
def calculate_air_volume_temp(T_Air_Volume_prev, dot_T_V_prev, time_delta):
    T_Air_Volume = T_Air_Volume_prev + dot_T_V_prev * time_delta
    return T_Air_Volume

In [ ]:
# dot_T_C
def calculate_dot_T_C(T_Air_Volume, T_Cargo):
    dot_T_C = alpha_c * (T_Air_Volume - T_Cargo)
    print("dot_T_C:", dot_T_C, "alpha_c:", alpha_c, "T_Air_Volume",T_Air_Volume, "T_Cargo:",T_Cargo)
    return dot_T_C

In [ ]:
# Calculate Term 7.1, Term 7.2, Term 7.3, Term 7.4
def calculate_eq_terms(alpha_tot, H_rad_tot, T_Ambient, T_Air_Volume, phi_U, phi_O, dot_T_C):
    term_7_1 = alpha_tot * (T_Ambient - T_Air_Volume)
    term_7_2 = H_rad_tot
    term_7_3 = (phi_U + phi_O) / sigma_v
    term_7_4 = -beta * dot_T_C
    return term_7_1, term_7_2, term_7_3, term_7_4

In [ ]:
# dot_T_V
def calculate_dot_T_V(term_7_1, term_7_2, term_7_3, term_7_4):
    return term_7_1 + term_7_2 + term_7_3 + term_7_4

# TRU

### TRU Coefficient of Performance

In [ ]:
def get_K_COP_values_at_temp(ideal_temperature):
    query_temp_key = str(5*round(ideal_temperature/5)) # Convert the temperature to nearest multiple of 5
    if ideal_temperature > 50 or ideal_temperature < -50:
        raise ValueError("Temperature range must be between -50C to 50C")

    t_min = -50           # in C
    t_max = 50            # in C
    K_COP_H_min = 0.80
    K_COP_H_max = 1.20
    K_COP_C_min = 1.20
    K_COP_C_max = 0.80
    temp_range = list(range(t_min, t_max+5, 5)) # values are [-50, -45,-40, ...... 45, 50]
    # Initial values of K_COP
    K_COP_H_i_prev = K_COP_H_min
    K_COP_C_i_prev = K_COP_C_min
    K_COP_dict = {}

    for idx, t in enumerate(temp_range):
        if t == t_min:
            K_COP_H = K_COP_H_min
            K_COP_C = K_COP_C_min
        elif t == t_max:
            K_COP_H = K_COP_H_max
            K_COP_C = K_COP_C_max
        else:
            t_i = t
            t_i_prev = temp_range[idx-1]
            K_COP_H = round(K_COP_H_i_prev+((t_i-t_i_prev)*(K_COP_H_max-K_COP_H_min)/(t_max-t_min)), 2)
            K_COP_C = round(K_COP_C_i_prev+((t_i-t_i_prev)*(K_COP_C_max-K_COP_C_min)/(t_max-t_min)), 2)
            # Keep the value as previous value for next iteration
            K_COP_H_i_prev = K_COP_H
            K_COP_C_i_prev = K_COP_C

        K_COP_dict[str(t)] = [K_COP_H, K_COP_C]

    # print(K_COP_dict)
    return K_COP_dict.get(query_temp_key)

In [ ]:
def get_K_COP(K_COP_H, K_COP_C):
    tru_mode = "Heating" if ideal_temperature>=temp_av else "Cooling"
    K_COP = K_COP_C if tru_mode=="Cooling" else K_COP_H
    return K_COP

In [ ]:
K_COP_H, K_COP_C = get_K_COP_values_at_temp(ideal_temperature) # Coefficient of Performance(COP) - Heating/ Cooling
K_COP = get_K_COP(K_COP_H, K_COP_C)                            # Coefficient of Performance(COP) - This case

print(K_COP_H, K_COP_C, K_COP)

0.88 1.12 1.12


### Fuel Cell Parameters

- TRU Heating/Cooling Output = **phi_on** W
- Fuel Cell efficiency = **fuel_cell_eff** (%)
- Fuel Cell Balance of Plant = **P_BOP** W

- Thermostat High Setting = **T_HI** C
- Thermostat Low Setting = **T_LO** C

In [ ]:
def calculate_phi_on():
    tru_mode = "Heating" if ideal_temperature>=temp_av else "Cooling"
    phi_on = compressor_power * K_COP_H if tru_mode=="Heating" else compressor_power * K_COP_C
    return phi_on

In [ ]:
phi_on = calculate_phi_on()                    # TRU Heating/Cooling Output W
fuel_cell_eff = fuel_cell_eff                  # Fuel Cell efficiency (%)
P_BOP = fuel_cell_bop                          # Fuel Cell Balance of Plant W
P_FAN = fuel_cell_fan                          # TRU fan power kW
T_HI = ideal_temperature + thermostat_range    # Thermostat High Setting C
T_LO = ideal_temperature - thermostat_range    # Thermostat Low Setting C

# Control Law

In [ ]:
def get_phi_current(phi_prev, T_V, mode):
    if mode == "Cooling":
        if phi_prev == 0:  # When TRU was Off
            if T_V <= T_HI:         # C1, case 1
                phi_current = 0
                N_case = "case 1"
            elif T_V > T_HI:        # !C1, case 3
                phi_current = phi_on
                N_case = "case 3"
            else:
                phi_current = "n/a"
                N_case = "case 9"
        else:    # When TRU was On
            if T_V <= T_LO:       # C2, case 2
                phi_current = 0
                N_case = "case 2"
            elif T_V > T_LO:        # !C2, case 4
                phi_current = phi_on
                N_case = "case 4"
            else:
                phi_current = "n/a"
                N_case = "case 10"
    elif mode == "Heating":  # When TRU is in Heating mode
        if phi_prev == 0:  # When TRU was Off
            if T_V <= T_LO:       # C2, case 6
                phi_current = -phi_on
                N_case = "case 6"
            elif T_V > T_LO:        # !C2, case 8
                phi_current = 0
                N_case = "case 8"
            else:
                phi_current = "n/a"
                N_case = "case 11"
        else:   # When TRU was On
            if T_V <= T_HI:         # C1, case 5
                phi_current = -phi_on
                N_case = "case 5"
            elif T_V > T_HI:        # !C1, case 7
                phi_current = 0
                N_case = "case 7"
            else:
                phi_current = "n/a"
                N_case = "case 12"
    else:
        phi_current = 0
        N_case = "case 0"

    return phi_current, N_case

# Model

In [ ]:
class ThermalSimulationModel:
    """ Simulated Values"""
    def get_ambient_temp(self, time_h):
        T_Ambient = calculate_ambient_temp(time_h)
        return T_Ambient

    def get_alpha_tot(self, time_h):
        alpha_tot = siml_radiation.get_alpha_tot(time_h)
        return alpha_tot

    def get_H_rad_tot(self, time_h):
        H_rad_tot = siml_radiation.get_H_rad_tot(time_h)
        return H_rad_tot

    """ Temperature Calculations"""
    def get_cargo_temp(self, T_Cargo_prev, dot_T_C, time_delta):
        T_Cargo = calculate_cargo_temp(T_Cargo_prev, dot_T_C, time_delta)
        return T_Cargo

    def get_air_volume_temp(self, T_Air_Volume_prev, dot_T_V, time_delta):
        T_Air_Volume = calculate_air_volume_temp(T_Air_Volume_prev, dot_T_V, time_delta)
        return T_Air_Volume

    def get_dot_T_C(self, T_Air_Volume_current, T_Cargo_current):
        dot_T_C = calculate_dot_T_C(T_Air_Volume_current, T_Cargo_current)
        return dot_T_C

    def get_eq_terms(self, alpha_tot, H_rad_tot, T_Ambient, T_Air_Volume, phi_U, phi_O, dot_T_C):
        term_7_1, term_7_2, term_7_3, term_7_4 = calculate_eq_terms(alpha_tot, H_rad_tot, T_Ambient,
                                                                    T_Air_Volume, phi_U, phi_O, dot_T_C)
        return term_7_1, term_7_2, term_7_3, term_7_4

    def get_dot_T_V(self, term_7_1, term_7_2, term_7_3, term_7_4):
        dot_T_V = calculate_dot_T_V(term_7_1, term_7_2, term_7_3, term_7_4)
        return dot_T_V


    """ Power Calculations
        phi_U = Active input from Heater/Cooler Unit
        phi_O = Active input from waste heat
    """
    def get_phi_U(self, T_Air_Volume, phi_U_prev):
        phi_U = get_phi_current(phi_U_prev, T_Air_Volume, TRU_MODE)
        return phi_U

    #def get_phi_O(self, phi_U, T_Ambient):
        #P_TOTAL = self.get_P_TOTAL(phi_U, T_Ambient)
        #phi_O = 0 if self.is_tru_mode_in_cooling() else P_TOTAL * (1-fuel_cell_eff)
        #return phi_O


    def get_phi_O(self, phi_U, T_Ambient):
         P_TOTAL = self.get_P_TOTAL(phi_U, T_Ambient)
         phi_O = 0 if TRU_MODE == 'Cooling' else P_TOTAL * (1-fuel_cell_eff)
         return phi_O

    def get_K_COP(self, T_Ambient):
        tru_mode = "Heating" if ideal_temperature>=T_Ambient else "Cooling"
        K_COP_H, K_COP_C = get_K_COP_values_at_temp(T_Ambient)
        K_COP = K_COP_C if tru_mode=="Cooling" else K_COP_H
        return K_COP

    #def get_P_U(self, phi_U, T_Ambient):
       # K_COP = self.get_K_COP(T_Ambient)
       # P_U = abs(phi_U) / K_COP
       # return P_U

    def get_P_U(self, phi_U, T_Ambient):
       K_COP = self.get_K_COP(T_Ambient)
       P_U = abs(phi_U[0]) / K_COP  # Assuming phi_U is a tuple and you want the first element
       return P_U

    def get_P_TRU(self, phi_U, T_Ambient):
        P_U = self.get_P_U(phi_U, T_Ambient)
        P_TRU = P_U + P_FAN
        return P_TRU

    def get_P_WORK(self, phi_U, T_Ambient):
        P_TRU = self.get_P_TRU(phi_U, T_Ambient)
        P_WORK = P_TRU + P_BOP
        return P_WORK

    def get_P_TOTAL(self, phi_U, T_Ambient):
        P_WORK = self.get_P_WORK(phi_U, T_Ambient)
        P_TOTAL = P_WORK / fuel_cell_eff
        return P_TOTAL

    def get_Power(self, T_Air_Volume, T_Ambient):
        phi_U = self.get_phi_U(T_Air_Volume)
        K_COP = self.get_K_COP(T_Ambient)
        Power = round(((abs(phi_U) / K_COP + P_FAN * 1000 + P_BOP) / fuel_cell_eff) / 1000, 2)
        return Power

    def get_Energy(self, energy_prev, power_prev):
        energy = energy_prev + power_prev * time_delta
        return energy

# Run Simulation

In [ ]:
model = ThermalSimulationModel()
number_of_data_points = 20

time_s_data = []
time_h_data = []
clock_h_data = []
ambient_temp_data = []
cargo_temp_data = []
air_vol_temp_data = []
alpha_tot_data = []
H_rad_tot_data = []
dot_T_C_data = []
dot_T_V_data = []
phi_U_data = []
phi_O_data = []
term_7_1_data = []
term_7_2_data = []
term_7_3_data = []
term_7_4_data = []



# Initial State
time_s = 0
time_h = 0
clock_h = init_clock_h
cargo_init_temp = T_LO                         # Cargo Load - Initial Temperature
air_volume_init_temp = temp_av                 # Air Volume Initial Temperature
T_Cargo_current = cargo_init_temp
T_Cargo_prev = None
T_Air_Volume_current = air_volume_init_temp
T_Air_Volume_prev = None
dot_T_V_prev = None
phi_U_current = 0
phi_U_prev = None


for i in range(number_of_data_points):
    time_s_data.append(time_s)
    time_h_data.append(time_h)
    clock_h_data.append(clock_h)

    """Simulated values"""
    alpha_tot = model.get_alpha_tot(time_h)
    alpha_tot_data.append(alpha_tot)
    H_rad_tot = model.get_H_rad_tot(time_h)
    H_rad_tot_data.append(H_rad_tot)
    # T_Ambient
    T_Ambient = model.get_ambient_temp(time_h)
    ambient_temp_data.append(T_Ambient)

    # T_Cargo
    if T_Cargo_prev is not None:
        T_Cargo_current = model.get_cargo_temp(T_Cargo_prev, dot_T_C, time_delta)
    cargo_temp_data.append(T_Cargo_current)    # append current value

    #T_Air_Volume
    if T_Air_Volume_prev is not None:
        T_Air_Volume_current = model.get_air_volume_temp(T_Air_Volume_prev, dot_T_V_prev, time_delta)
    air_vol_temp_data.append(T_Air_Volume_current)

    """ Active input from Heater/Cooler Unit"""
    phi_U_current = model.get_phi_U(T_Air_Volume_current, phi_U_prev)
    phi_U_data.append(phi_U_current)

    """ Active input from waste heat"""
    phi_O = model.get_phi_O(phi_U_current, T_Ambient)
    phi_O_data.append(phi_O)


    """" Temperatures"""
    # dot_T_C
    dot_T_C = model.get_dot_T_C(T_Air_Volume_current, T_Cargo_current)
    dot_T_C_data.append(dot_T_C)

    # Equation 7 terms
    term_7_1, term_7_2, term_7_3, term_7_4 = model.get_eq_terms(alpha_tot,
                                                                H_rad_tot, T_Ambient, T_Air_Volume_current,
                                                                phi_U_current[0], phi_O, dot_T_C)
    term_7_1_data.append(term_7_1)
    term_7_2_data.append(term_7_2)
    term_7_3_data.append(term_7_3)
    term_7_4_data.append(term_7_4)

    # dot_T_V
    dot_T_V_current = model.get_dot_T_V(term_7_1, term_7_2, term_7_3, term_7_4)
    dot_T_V_data.append(dot_T_V_current)



    # Update clock for next iteration
    time_s, time_h, clock_h = clck.update_clock_time(time_s)
    # update values for next iteration
    T_Cargo_prev = T_Cargo_current
    # T_Air_Volume
    T_Air_Volume_prev = T_Air_Volume_current
    # dot_T_V
    dot_T_V_prev = dot_T_V_current



data = {
    'time_s': time_s_data,
    'time_h': time_h_data,
    'clock_h': clock_h_data,
    'T_Ambient': ambient_temp_data,
    'T_Cargo': cargo_temp_data,
    'T_Air_Volume':  air_vol_temp_data,
    'alpha_tot': alpha_tot_data,
    'H_rad_tot': H_rad_tot_data,
    'dot_T_C': dot_T_C_data,
    'dot_T_V': dot_T_V_data,
    'phi_U': phi_U_data,
    'phi_O': phi_O_data,
    'Term 7.1':term_7_1_data,
    'Term 7.2':term_7_2_data,
    'Term 7.3':term_7_3_data,
    'Term 7.4':term_7_4_data

}

df = pd.DataFrame(data)
df
# print(data )


dot_T_C: 0.006371252204585538 alpha_c: 8.495002939447384e-05 T_Air_Volume 42 T_Cargo: -33
dot_T_C: 0.004055791656281768 alpha_c: 8.495002939447384e-05 T_Air_Volume 14.775119979606234 T_Cargo: -32.96814373897707
dot_T_C: 0.0026116075723400345 alpha_c: 8.495002939447384e-05 T_Air_Volume -2.2049964031842535 T_Cargo: -32.94786478069566
dot_T_C: 0.0017108499646649798 alpha_c: 8.495002939447384e-05 T_Air_Volume -12.795320307573544 T_Cargo: -32.93480674283396
dot_T_C: 0.0011490345265451499 alpha_c: 8.495002939447384e-05 T_Air_Volume -19.40024744350683 T_Cargo: -32.92625249301064
dot_T_C: 0.0007986217331686587 alpha_c: 8.495002939447384e-05 T_Air_Volume -23.51943072439252 T_Cargo: -32.92050732037791
dot_T_C: 0.0005800635221397215 alpha_c: 8.495002939447384e-05 T_Air_Volume -26.088223200226484 T_Cargo: -32.91651421171206
dot_T_C: 0.0004437447310978304 alpha_c: 8.495002939447384e-05 T_Air_Volume -27.690016748098532 T_Cargo: -32.91361389410137
dot_T_C: 0.00035871973720918483 alpha_c: 8.4950029394

,time_s,time_h,clock_h,T_Ambient,T_Cargo,T_Air_Volume,alpha_tot,H_rad_tot,dot_T_C,dot_T_V,phi_U,phi_O,Term 7.1,Term 7.2,Term 7.3,Term 7.4
0,0,0.00,9.00,42.0,-33.000000,42.000000,0.00257,0.0,0.006371,-5.444976,"(16.8, case 4)",0,0.000000,0.0,0.000406,-5.445382
1,5,0.00,9.00,42.0,-32.968144,14.775120,0.00257,0.0,0.004056,-3.396023,"(16.8, case 4)",0,0.069975,0.0,0.000406,-3.466404
2,10,0.00,9.00,42.0,-32.947865,-2.204996,0.00257,0.0,0.002612,-2.118065,"(16.8, case 4)",0,0.113618,0.0,0.000406,-2.232089
3,15,0.00,9.00,42.0,-32.934807,-12.795320,0.00257,0.0,0.001711,-1.320985,"(16.8, case 4)",0,0.140838,0.0,0.000406,-1.462229
4,20,0.01,9.01,42.0,-32.926252,-19.400247,0.00257,0.0,0.001149,-0.823837,"(16.8, case 4)",0,0.157814,0.0,0.000406,-0.982057
5,25,0.01,9.01,42.0,-32.920507,-23.519431,0.00257,0.0,0.000799,-0.513758,"(16.8, case 4)",0,0.168402,0.0,0.000406,-0.682566
6,30,0.01,9.01,42.0,-32.916514,-26.088223,0.00257,0.0,0.000580,-0.320359,"(16.8, case 4)",0,0.175004,0.0,0.000406,-0.495769
7,35,0.01,9.01,42.0,-32.913614,-27.690017,0.00257,0.0,0.000444,-0.199733,"(16.8, case 4)",0,0.179121,0.0,0.000406,-0.379260
8,40,0.01,9.01,42.0,-32.911395,-28.688680,0.00257,0.0,0.000359,-0.124497,"(16.8, case 4)",0,0.181688,0.0,0.000406,-0.306591
9,45,0.01,9.01,42.0,-32.909602,-29.311164,0.00257,0.0,0.000306,-0.077571,"(16.8, case 4)",0,0.183288,0.0,0.000406,-0.261265


# Dashboard

### Calculations

In [ ]:
def get_phi_predicted(tru_mode, heating_demand_with_waste_heat, phi_baseline):
    if tru_mode == "Heating":
        if is_waste_heat_used == True:
            phi_H_predicted = heating_demand_with_waste_heat
        else:
            phi_H_predicted = phi_baseline
        return round(phi_H_predicted/1000, 2)
    elif tru_mode == "Cooling":
        return round(phi_baseline/1000, 2)

    return 0

In [ ]:
T_A_ideal = ideal_temperature	                     # Ideal Operating Temperature for Cargo in C
temp_diff_heating = temp_lo - ideal_temperature      # Temperature difference (worst case - heating) C
temp_diff_cooling = temp_hi - ideal_temperature      # Temperature difference (worst case - cooling) C
# sigma_v = Sigma V

"""Logical Inconsistancy. Can not calculate the average Alpha & Radiation for the whole duration
in the middle of the mission"""
# alpha_tot_avg= AVERAGE('Sun & Road'!alpha_tot)                                 # Alpha total (average)
# H_rad_tot_avg = AVERAGE('Sun & Road'!H_rad_tot)                                # Radiation total (averge)
# phi_baseline = -sigma_v * (temp_diff_cooling * alpha_tot_avg + H_rad_tot_avg)  # Baseline heating/cooling demand W
# baseline_mode = "Heating" if phi_baseline>=0 else "Cooling"                    # Baseline mode

# heating_eff_h = round(fuel_cell_eff * K_COP_H/(1 - fuel_cell_eff + fuel_cell_eff * K_COP_H), 2) # Heating Efficiency
# heating_demand_with_waste_heat = -heating_eff_h * (-phi_baseline +
#                                                    (1-fuel_cell_eff)* (P_BOP + P_FAN)/fuel_cell_eff) # Heating demand with waste heat W

# phi_H_predicted = get_phi_predicted(get_tru_mode(),
#                                       heating_demand_with_waste_heat,
#                                       phi_baseline) # Predicted Heating Demand kW

# phi_C_predicted = get_phi_predicted(get_tru_mode(),
#                                       heating_demand_with_waste_heat,
#                                       phi_baseline) # Predicted Heating Demand kW





'Logical Inconsistancy. Can not calculate the average Alpha & Radiation for the whole duration \nin the middle of the mission'